In [2]:
import numpy as np
import pandas as pd
from create_model_features import trends_features
from copy import deepcopy
import datetime
################## INSERT CCY You want to RUN
ccy = "EURUSD"
ccy1 = ccy[:3]
ccy2 = ccy[3:]
edi_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\EDI.csv"
esi_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\ESI.csv"
features = ["EDI_"+ccy1 + ccy2 + "_spread", "EDI_G10_normalised", "EDI_Global_normalised",
            "ESI_"+ccy1 + ccy2 + "_spread", "ESI_G10_normalised", "ESI_Global_normalised"]
EDI = pd.read_csv(edi_location)
ESI = pd.read_csv(esi_location)
EDI['Date'] =  pd.to_datetime(EDI['Date'], format= '%d/%m/%Y')
ESI['Date'] =  pd.to_datetime(ESI['Date'], format= '%d/%m/%Y')
EDI['Date'] =  EDI['Date'].dt.date
ESI['Date'] =  ESI['Date'].dt.date
EDI["EDI_"+ ccy1 + ccy2 + "_spread"] = EDI["EDI_" +ccy1 + "_normalised"] - EDI["EDI_" +ccy2 + "_normalised"]
ESI["ESI_"+ ccy1 + ccy2 + "_spread"] = ESI["ESI_" +ccy1 + "_normalised"] - ESI["ESI_" + ccy2 + "_normalised"]
EDI_tradable = EDI[["EDI_"+ccy1 + ccy2 + "_spread", "EDI_G10_normalised", "EDI_Global_normalised"]].cumsum()
ESI_tradable = ESI[["ESI_"+ccy1 + ccy2 + "_spread", "ESI_G10_normalised", "ESI_Global_normalised"]].cumsum()
EDI_tradable['Date'] = EDI['Date']
ESI_tradable['Date'] = ESI['Date']
short, medium, long, longest, medium_multiplier,long_multplier = 21, 55, 100, 200, 1, 1 # as all in days.
for col in features:
    if col in list(EDI_tradable.columns):
        data = deepcopy(EDI_tradable)
        data = trends_features(data,col, short, medium, long,longest,medium_multiplier,long_multplier)
        EDI_tradable[col+"_spotvma"] = data['spot_v_HF']
        EDI_tradable[col+"_madiff"] = data['HF_ema_diff']
    if col in list(ESI_tradable.columns):
        data = deepcopy(ESI_tradable)
        data = trends_features(data,col, short, medium, long, longest,medium_multiplier,long_multplier)
        ESI_tradable[col+"_spotvma"] = data['spot_v_HF']
        ESI_tradable[col+"_madiff"] = data['HF_ema_diff']
print(EDI_tradable.columns)
signal_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\best_results\Signals.csv"
signals = pd.read_csv(signal_location)
signals['Date'] = pd.to_datetime(signals['Date'], format= '%d/%m/%Y %H:%M')
signals['DDMMYY'] = signals['Date'].dt.date # signals['Date'].apply(get_ddmmyy)
#signals['DDMMYY'] = pd.to_datetime(signals['DDMMYY'])
trend_estimation = signals.groupby('DDMMYY').sum().reset_index(drop = False)
# i.e. the average of the ccy kevel across each hour
trend_estimation['CCY'] = signals.groupby('DDMMYY').mean().reset_index(drop = False)['CCY']
# overwriting the dte arg as its easier to understand
trend_estimation['Date'] = trend_estimation['DDMMYY']
trend_estimation[ccy1+ccy2 +"_vol"] = trend_estimation['CCY'].rolling(60).std()*np.sqrt(260)
trend_estimation = pd.merge(trend_estimation,EDI_tradable,how="left" ,on= "Date")
trend_estimation = pd.merge(trend_estimation,ESI_tradable,how="left" ,on= "Date")
trend_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\best_results\trendestimation.csv"
trend_estimation.replace(np.nan,0).to_csv(trend_location)

Index(['EDI_EURUSD_spread', 'EDI_G10_normalised', 'EDI_Global_normalised',
       'Date', 'EDI_EURUSD_spread_spotvma', 'EDI_EURUSD_spread_madiff',
       'EDI_G10_normalised_spotvma', 'EDI_G10_normalised_madiff',
       'EDI_Global_normalised_spotvma', 'EDI_Global_normalised_madiff'],
      dtype='object')


In [31]:
from matplotlib import pyplot as plt
# model features
features = ["EDI_"+ccy1 + ccy2 + "_spread", "EDI_G10_normalised", "EDI_Global_normalised",
            "ESI_"+ccy1 + ccy2 + "_spread", "ESI_G10_normalised", "ESI_Global_normalised"]

trends_features(EDI_tradable,"EDI_G10_normalised", short, medium, long, longest,medium_multiplier,
                long_multplier)['spot_v_HF'].values

array([  0.        ,  -0.39104105,  -0.25810771, ...,  -9.83269382,
       -10.32362597,  -8.81085717])

In [38]:
x = signals['Date'].loc[1]
#datetime.datetime.strptime(x,"%d/%m/%y HH:MM")
type(x)
#datetime.datetime.strftime(x,"%d/%m/%YY")

str